# Semtiment Analysis Model

Import packages

In [1]:
import numpy as np
import pandas as pd
import re

Load dataset downloaded from https://www.kaggle.com/paoloripamonti/twitter-sentiment-analysis/data

In [2]:
sentiment_df = pd.read_csv(
    "database/sentiment_training.zip",
    header=None,
    names=["emotion", "text"],
    usecols=[0, 5],
    encoding = "ISO-8859-1"
)

sentiment_df["emotion"] = sentiment_df["emotion"].apply(lambda x: x // 2)
sentiment_df = sentiment_df[["text", "emotion"]]

Clean then tokenize

In [37]:
def clean_text(text):
    text = re.compile("(@[\w]+|https?://[\w\./]+|)").sub("", text) # Remove @ and urls, keep hashtags
    text = re.compile("[^\w]+ ").sub(" ", text) # Remove puncutations
    #text = (
    #    re
    #    .compile("( the | a | an | is | are | was | were | this | that | these | those |'m |'s |'re )")
    #    .sub(" ", text) # Remove stopwords
    #)
    return text.lower().strip()

In [38]:
sentiment_df = sentiment_df.assign(tokens=sentiment_df["text"].apply(clean_text).apply(lambda text: text.split(" ")))

In [40]:
pd.Series(sentiment_df["tokens"].head(10000).aggregate("sum")).value_counts().head(30)

i        6197
to       4046
my       2265
it       1957
and      1938
in       1541
for      1338
of       1267
you      1127
but      1021
on        997
have      996
me        993
not       965
so        936
at        832
just      814
be        730
work      713
up        690
with      638
no        635
now       598
get       565
go        535
out       531
all       509
today     494
do        477
too       475
dtype: int64